In [1]:
import argparse
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
mpl.use("Agg")

import sys
sys.path.append('..')

from visualizations import translation
from visualizations import scale
from visualizations import inversion
from visualizations.utils import default_parser

In [2]:
def create_args_string(kwargs, flags):
    s = ""
    for k, v in kwargs.items():
        s += f"--{k}={v} "
    for f in flags:
        s += f"--{f} "
    return s

# Translation Symmetry

### VGG16

In [7]:
args = {
    "save-dir": "/mnt/fs4/kunin/neural-mechanics/results",
    "experiment": "vgg16",
    "expid": "lr_0.1_wd_0.0",
    "normalize": True,
}
flags = []
parser = default_parser()
parser = translation.extend_parser(parser)
ARGS = parser.parse_args(create_args_string(args, flags).split())
translation.main(ARGS)

>> Loading weights...


AttributeError: module 'utils' has no attribute 'makedir_quiet'

In [6]:
print(utils)

NameError: name 'utils' is not defined

### VGG16-BN

# Scale Symmetry

### VGG16-BN

# Inversion Symmetry

### VGG16-BN